In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MultiLabelBinarizer

In [25]:
# data from 
    # https://www.kaggle.com/datasets/smvjkk/league-of-legends-lec-summer-playoffs-2024-stats
    # https://www.kaggle.com/datasets/cutedango/league-of-legends-champions

In [26]:
# Read the files
play_offs = pd.read_csv('LEC_SummerPlayoffs_2024.csv')
champions = pd.read_csv('LOL_champions.csv')

In [27]:
# Dictionaries
role_dict = {'TOP':'Top',
             'JUNGLE':'Jungle',
             'MID':'Middle',
             'ADCARRY':'Bottom',
             'SUPPORT':'Support'}

champion_dict = {'Aurelion_Sol':"Aurelion Sol",
                 'BelVeth':"Bel'Veth",
                 'ChoGath':"Cho'Gath",
                 'Dr._Mundo':"Dr. Mundo",
                 'Jarvan_IV':"Jarvan IV",
                 'KaiSa':"Kai'Sa",
                 'KhaZix':"Kha'Zix",
                 'KogMaw':"Kog'Maw", 
                 'KSante':"K'Sante",
                 'Lee_Sin':"Lee Sin",
                 'Master_Yi':"Master Yi",
                 'Miss_Fortune':"Miss Fortune",
                 'Nunu_&_Willump':"Nunu & Willump",
                 'RekSai':"Rek'Sai",
                 'Renata_Glasc':"Renata Glasc",
                 'Tahm_Kench':"Tahm Kench",
                 'Twisted_Fate':"Twisted Fate",
                 'VelKoz':"Vel'Koz",
                 'Xin_Zhao':"Xin Zhao"}

# Data Cleaning

In [28]:
play_offs.head()

,Player,Role,Team,Opponent_Team,Opponent_Player,Date,Round,Day,Patch,Stage,...,Damage self mitigated,Total Damage Shielded On Teammates,Time ccing others,Total Time CC Dealt,Total damage taken,Total Time Spent Dead,Consumables purchased,Items Purchased,Shutdown bounty collected,Shutdown bounty lost
0,Irrelevant,TOP,SK,TH,Wunder,7/12/2024,1,1,v14.13,Summer_Playoffs_24,...,9432,0,17,361,13153,0,4,16,50,50
1,Isma,JUNGLE,SK,TH,Jankos,7/12/2024,1,1,v14.13,Summer_Playoffs_24,...,15538,0,8,204,27102,0,5,19,0,0
2,Nisqy,MID,SK,TH,Zwyroo,7/12/2024,1,1,v14.13,Summer_Playoffs_24,...,7856,0,4,144,14367,36,8,24,0,100
3,Rahel,ADCARRY,SK,TH,Flakked,7/12/2024,1,1,v14.13,Summer_Playoffs_24,...,4962,0,2,116,8087,0,4,19,0,0
4,Luon,SUPPORT,SK,TH,Trymbi,7/12/2024,1,1,v14.13,Summer_Playoffs_24,...,5672,0,13,37,5643,10,14,25,0,0


In [29]:
play_offs.size

34020

In [30]:
# Dropping columns and replacing rows
play_offs.drop(labels=['Date', 'Patch', 'Stage'], axis=1, inplace=True, errors='ignore')
play_offs['Role'] = play_offs['Role'].replace(role_dict)

In [31]:
# Renaming columns
champions.rename(columns={'Name':'Champion'}, inplace=True)

champions

,Champion,Tags,Role,Range type,Resourse type,Base HP,HP per lvl,Base mana,Mana per lvl,Movement speed,...,Attack range,HP regeneration,HP regeneration per lvl,Mana regeneration,Mana regeneration per lvl,Attack damage,Attack damage per lvl,Attack speed per lvl,Attack speed,AS ratio
0,Aatrox,Fighter,Top,Melee,Blood Well,650,114,0,0.0,345,...,175,3.00,0.50,0.00,0.00,60,5.00,2.500,0.651,0.651
1,Ahri,"Mage,Assassin",Middle,Ranged,Mana,590,104,418,25.0,330,...,550,2.50,0.60,8.00,0.80,53,3.00,2.200,0.668,0.625
2,Akali,Assassin,"Top,Middle",Melee,Energy,600,119,200,0.0,345,...,125,9.00,0.90,50.00,0.00,62,3.30,3.200,0.625,0.625
3,Akshan,"Marksman,Assassin",Middle,Ranged,Mana,630,107,350,40.0,330,...,500,3.75,0.65,8.20,0.70,52,3.00,4.000,0.638,0.400
4,Alistar,"Tank,Support",Support,Melee,Mana,685,120,350,40.0,330,...,125,8.50,0.85,8.50,0.80,62,3.75,2.125,0.625,0.625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162,Zeri,Marksman,Bottom,Ranged,Mana,600,110,250,45.0,330,...,500,3.25,0.70,6.00,0.80,56,2.00,2.000,0.658,0.625
163,Ziggs,Mage,"Bottom,Middle",Ranged,Mana,606,106,480,23.5,325,...,550,6.50,0.60,8.00,0.80,55,3.10,2.000,0.656,0.656
164,Zilean,"Support,Mage",Support,Ranged,Mana,574,96,452,50.0,335,...,550,5.50,0.50,11.35,0.80,52,3.00,2.130,0.658,0.625
165,Zoe,Mage,Middle,Ranged,Mana,630,106,425,25.0,340,...,550,7.50,0.60,8.00,0.65,58,3.30,2.500,0.658,0.625


# Creating columns and a cleaner data set

In [32]:
# Create a helper column to group teams: every 5 rows represents a team
play_offs['Team_ID'] = np.repeat(range(len(play_offs) // 5), 5)

# Renaming columns
play_offs.rename(columns={'Outcome':'Win'}, inplace = True)

# Changing rows
# string values to binary values
play_offs['Win'] = play_offs['Win'].map({'Win': 1, 'Loss': 0})

In [33]:
# Changing rows
# names to formal names
play_offs['Champion'] = play_offs['Champion'].apply(lambda x: champion_dict.get(x, x))
play_offs['Champion_Opponent'] = play_offs['Champion_Opponent'].apply(lambda x: champion_dict.get(x, x))

In [34]:
# Use only the necessary columns
play_offs_clean = play_offs[['Champion', 'Champion_Opponent','Win', 'Team_ID']]
play_offs_clean

,Champion,Champion_Opponent,Win,Team_ID
0,Gnar,Twisted Fate,1,0
1,Nidalee,Ivern,1,0
2,Tristana,Varus,1,0
3,Zeri,Smolder,1,0
4,Rell,Leona,1,0
...,...,...,...,...
415,Ornn,Renekton,1,83
416,Zyra,Brand,1,83
417,Corki,Lucian,1,83
418,Kaisa,Ezreal,1,83


In [35]:
# Group by team_id and collect all champions for each team
team_comps = play_offs_clean.groupby('Team_ID').agg({
    'Champion': lambda champs: tuple(sorted(champs)),
    'Champion_Opponent': lambda champs: tuple(sorted(champs))
    }).reset_index()
team_comps.rename(columns = {'Champion':'Comp', 'Champion_Opponent':'Comp_Opponent'}, inplace = True)
team_comps

,Team_ID,Comp,Comp_Opponent
0,0,"(Gnar, Nidalee, Rell, Tristana, Zeri)","(Ivern, Leona, Smolder, Twisted Fate, Varus)"
1,1,"(Ivern, Leona, Smolder, Twisted Fate, Varus)","(Gnar, Nidalee, Rell, Tristana, Zeri)"
2,2,"(Braum, Camille, Taliyah, Xin Zhao, Zeri)","(Gnar, Kaisa, Rell, Vi, Ziggs)"
3,3,"(Gnar, Kaisa, Rell, Vi, Ziggs)","(Braum, Camille, Taliyah, Xin Zhao, Zeri)"
4,4,"(Corki, Gnar, Nautilus, Taliyah, Twitch)","(Braum, Camille, Tristana, Zeri, Zyra)"
...,...,...,...
79,79,"(Corki, Ezreal, Kennen, Poppy, Sejuani)","(Jhin, Lucian, Rell, Renekton, Zyra)"
80,80,"(Ezreal, Leona, Lucian, Renekton, Zyra)","(Brand, Corki, Jhin, Nautilus, Ornn)"
81,81,"(Brand, Corki, Jhin, Nautilus, Ornn)","(Ezreal, Leona, Lucian, Renekton, Zyra)"
82,82,"(Brand, Ezreal, Leona, Lucian, Renekton)","(Corki, Kaisa, Nautilus, Ornn, Zyra)"


In [36]:
# Merge the outcome back into team compositions based on the team_id
team_comps = team_comps.merge(play_offs_clean[['Team_ID', 'Win']].drop_duplicates(), on='Team_ID')

# Now you can calculate win rates by grouping by team composition
win_rates = team_comps.groupby('Comp')['Win'].mean()

In [37]:
team_comps.drop(labels={'Win_x', 'Win_y'}, axis=1, inplace=True, errors='ignore')
team_comps

,Team_ID,Comp,Comp_Opponent,Win
0,0,"(Gnar, Nidalee, Rell, Tristana, Zeri)","(Ivern, Leona, Smolder, Twisted Fate, Varus)",1
1,1,"(Ivern, Leona, Smolder, Twisted Fate, Varus)","(Gnar, Nidalee, Rell, Tristana, Zeri)",0
2,2,"(Braum, Camille, Taliyah, Xin Zhao, Zeri)","(Gnar, Kaisa, Rell, Vi, Ziggs)",0
3,3,"(Gnar, Kaisa, Rell, Vi, Ziggs)","(Braum, Camille, Taliyah, Xin Zhao, Zeri)",1
4,4,"(Corki, Gnar, Nautilus, Taliyah, Twitch)","(Braum, Camille, Tristana, Zeri, Zyra)",0
...,...,...,...,...
79,79,"(Corki, Ezreal, Kennen, Poppy, Sejuani)","(Jhin, Lucian, Rell, Renekton, Zyra)",1
80,80,"(Ezreal, Leona, Lucian, Renekton, Zyra)","(Brand, Corki, Jhin, Nautilus, Ornn)",0
81,81,"(Brand, Corki, Jhin, Nautilus, Ornn)","(Ezreal, Leona, Lucian, Renekton, Zyra)",1
82,82,"(Brand, Ezreal, Leona, Lucian, Renekton)","(Corki, Kaisa, Nautilus, Ornn, Zyra)",0


In [39]:
#match_history = {

# Similar champions

In [40]:
# Combine aspects into a tuple
champions['Role Tags Combo'] = champions[['Role', 'Tags']].apply(tuple, axis=1)

In [41]:
#Group by the aspect combinations and aggregate substitute champions into lists
champions_sub = champions.groupby(['Role', 'Tags'])['Champion'].apply(list).to_dict()
champions_sub

{('Bottom', 'Fighter,Assassin'): ['Nilah'],
 ('Bottom', 'Marksman'): ['Aphelios',
  'Caitlyn',
  'Draven',
  'Jinx',
  'Kalista',
  'Sivir',
  'Xayah',
  'Zeri'],
 ('Bottom', 'Marksman,Assassin'): ['Lucian', 'Samira'],
 ('Bottom', 'Marksman,Mage'): ['Ezreal',
  'Jhin',
  "Kai'Sa",
  "Kog'Maw",
  'Miss Fortune',
  'Varus'],
 ('Bottom,Middle', 'Mage'): ['Ziggs'],
 ('Bottom,Middle', 'Marksman,Assassin'): ['Tristana'],
 ('Bottom,Support', 'Marksman,Assassin'): ['Twitch'],
 ('Bottom,Support', 'Marksman,Support'): ['Ashe'],
 ('Bottom,Support', 'Support,Mage'): ['Seraphine'],
 ('Bottom,Support', 'Support,Marksman'): ['Senna'],
 ('Bottom,Top', 'Marksman,Assassin'): ['Vayne'],
 ('Bottom,Top,Middle', 'Marksman,Mage'): ['Smolder'],
 ('Jungle', 'Assassin'): ["Kha'Zix"],
 ('Jungle', 'Assassin,Fighter'): ['Master Yi'],
 ('Jungle', 'Assassin,Mage'): ['Elise', 'Evelynn', 'Nidalee'],
 ('Jungle', 'Fighter'): ["Bel'Veth"],
 ('Jungle', 'Fighter,Assassin'): ['Briar',
  'Kayn',
  'Lee Sin',
  'Nocturne',
  

In [42]:
champions

,Champion,Tags,Role,Range type,Resourse type,Base HP,HP per lvl,Base mana,Mana per lvl,Movement speed,...,HP regeneration,HP regeneration per lvl,Mana regeneration,Mana regeneration per lvl,Attack damage,Attack damage per lvl,Attack speed per lvl,Attack speed,AS ratio,Role Tags Combo
0,Aatrox,Fighter,Top,Melee,Blood Well,650,114,0,0.0,345,...,3.00,0.50,0.00,0.00,60,5.00,2.500,0.651,0.651,"(Top, Fighter)"
1,Ahri,"Mage,Assassin",Middle,Ranged,Mana,590,104,418,25.0,330,...,2.50,0.60,8.00,0.80,53,3.00,2.200,0.668,0.625,"(Middle, Mage,Assassin)"
2,Akali,Assassin,"Top,Middle",Melee,Energy,600,119,200,0.0,345,...,9.00,0.90,50.00,0.00,62,3.30,3.200,0.625,0.625,"(Top,Middle, Assassin)"
3,Akshan,"Marksman,Assassin",Middle,Ranged,Mana,630,107,350,40.0,330,...,3.75,0.65,8.20,0.70,52,3.00,4.000,0.638,0.400,"(Middle, Marksman,Assassin)"
4,Alistar,"Tank,Support",Support,Melee,Mana,685,120,350,40.0,330,...,8.50,0.85,8.50,0.80,62,3.75,2.125,0.625,0.625,"(Support, Tank,Support)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162,Zeri,Marksman,Bottom,Ranged,Mana,600,110,250,45.0,330,...,3.25,0.70,6.00,0.80,56,2.00,2.000,0.658,0.625,"(Bottom, Marksman)"
163,Ziggs,Mage,"Bottom,Middle",Ranged,Mana,606,106,480,23.5,325,...,6.50,0.60,8.00,0.80,55,3.10,2.000,0.656,0.656,"(Bottom,Middle, Mage)"
164,Zilean,"Support,Mage",Support,Ranged,Mana,574,96,452,50.0,335,...,5.50,0.50,11.35,0.80,52,3.00,2.130,0.658,0.625,"(Support, Support,Mage)"
165,Zoe,Mage,Middle,Ranged,Mana,630,106,425,25.0,340,...,7.50,0.60,8.00,0.65,58,3.30,2.500,0.658,0.625,"(Middle, Mage)"


# Models

In [43]:
# One-hot encode team compositions
#mlb = MultiLabelBinarizer()
#team_comps_encoded = mlb.fit_transform(team_comps['Comp'])
# Now team_comps_encoded is a binary matrix you can use to train a model

# Implementation

In [44]:
user_preferences = {
    'Top': ['Fiora', 'Ornn'],        # List of champions for the Top lane
    'Jungle': ['Elise', 'Lee Sin'],  # List of champions for the Jungle role
    'Mid': ['Syndra', 'LeBlanc'],    # List of champions for the Mid lane
    'Bottom': ['Jinx', 'Jhin'],      # List of champions for the Bottom role
    'Support': ['Leona', 'Thresh']   # List of champions for the Support role
}

In [45]:
def get_substitute_champion(role, tags, preferred_champions, champion_substitution):
    available_champions = champion_substitution.get((role, tags), [])
    
    # Check if any user-preferred champions are available
    for champion in available_champions:
        if champion in preferred_champions:
            return champion
    
    # If no preferred champion is available, return the best option from available champions
    return available_champions if available_champions else None

In [46]:
get_substitute_champion('Support', 'Tank,Support', user_preferences['Support'], champions_sub)

'Leona'

In [47]:
def suggest_best_team_with_substitution(user_preferences, match_history, champion_substitution):
    from itertools import product

    possible_compositions = []
    
    for role, champions in user_preferences.items():
        # Get possible substitutes for each role
        role_tags = [(r, t) for (r, t) in champion_substitution if r == role]
        substitutes = [get_substitute_champion(r, t, champions, champion_substitution) for (r, t) in role_tags]
        possible_compositions.append(champions + substitutes)

    # Create combinations and evaluate win rates
    all_combinations = list(product(*possible_compositions))
    
    best_comp = None
    best_win_rate = 0
    
    for comp in all_combinations:
        win_rate = get_win_rate(comp, match_history)
        if win_rate and win_rate > best_win_rate:
            best_win_rate = win_rate
            best_comp = comp
    
    return best_comp, best_win_rate

In [49]:
# suggest_best_team_with_substitution(user_preferences